In [10]:
# Import necessary libraries
import pandas as pd
import yfinance as yf
import datetime as dt

In [ ]:
# Get S&P 500 tickers
sp500_tickers = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
sp500_tickers = sp500_tickers.Symbol.to_list()
sp500_tickers

In [ ]:
# Clean Tickers of periods
sp500_tickers = [x.replace(".", "-") for x in sp500_tickers]
sp500_tickers

In [38]:
# Define Start and End dates for hist. data(5 yr)
end_date = dt.datetime.now().date().isoformat()
start_date = dt.datetime.now() - dt.timedelta(weeks=52*5)
start_date = start_date.date().isoformat()
start_date

'2016-08-17'

In [107]:
sp_data = yf.download(sp500_tickers, period="5y", groupby="Ticker")
sp_data

[*********************100%***********************]  505 of 505 completed


Adj Close                                                 \
                     A        AAL         AAP        AAPL        ABBV   
Date                                                                    
2016-08-11         NaN        NaN         NaN         NaN         NaN   
2016-08-12   46.114536  33.659374  162.385315   25.401896   52.770931   
2016-08-15   46.401909  34.575081  164.106995   25.707157   52.928009   
2016-08-16   46.248650  35.423321  156.907288   25.683668   52.417500   
2016-08-17   46.392342  35.278732  154.608459   25.646103   52.488182   
...                ...        ...         ...         ...         ...   
2021-08-05  155.039993  20.910000  206.339996  147.059998  115.120003   
2021-08-06  155.699997  21.020000  205.720001  146.139999  114.449997   
2021-08-09  156.539993  20.559999  204.520004  146.089996  114.059998   
2021-08-10  156.520004  20.940001  208.740005  145.600006  114.820000   
2021-08-11  156.520004  21.230000  211.210007  145.860001  113.720001   

                                                                        ...  \
                   ABC        ABMD         ABT         ACN        ADBE  ...   
Date                                                                    ...   
2016-08-11         NaN         NaN         NaN         NaN         NaN  ...   
2016-08-12   81.435448  123.690002   41.121891  103.718346  101.040001  ...   
2016-08-15   81.875076  123.389999   41.222328  104.241379  100.930000  ...   
2016-08-16   81.939201  122.459999   41.012329  103.672455  100.599998  ...   
2016-08-17   81.462921  121.760002   40.793205  103.626564   99.699997  ...   
...                ...         ...         ...         ...         ...  ...   
2021-08-05  121.889999  348.859985  122.209999  320.309998  632.080017  ...   
2021-08-06  120.639999  345.880005  122.169998  321.829987  631.380005  ...   
2021-08-09  120.230003  346.649994  123.160004  319.519989  629.219971  ...   
2021-08-10  118.970001  332.700012  122.699997  320.179993  621.739990  ...   
2021-08-11  119.239998  319.880005  121.459999  319.790009  626.030029  ...   

                 Volume                                                   \
                    XEL         XLNX           XOM       XRAY        XYL   
Date                                                                       
2016-08-11          NaN          NaN           NaN        NaN        NaN   
2016-08-12    1907300.0    1151900.0  9.986900e+06   802100.0  1224000.0   
2016-08-15    2156200.0    1702400.0  6.432600e+06   814200.0  1818500.0   
2016-08-16    3580700.0    1182100.0  6.817900e+06   972800.0  1950500.0   
2016-08-17    5176600.0    1568900.0  7.104600e+06  1481100.0   901300.0   
...                 ...          ...           ...        ...        ...   
2021-08-05  202570000.0  588700000.0  1.517250e+09  5220000.0   698200.0   
2021-08-06  207730000.0  347310000.0  1.488020e+09  1743000.0   763400.0   
2021-08-09  141510000.0  387410000.0  1.781300e+09  1845200.0   516400.0   
2021-08-10  157910000.0  203970000.0  1.632820e+09  1640000.0   646500.0   
2021-08-11    1728896.0    2529353.0  1.712002e+07   722473.0   551184.0   

                                                                      
                  YUM        ZBH      ZBRA         ZION          ZTS  
Date                                                                  
2016-08-11        NaN        NaN       NaN          NaN          NaN  
2016-08-12  3141574.0   867200.0  852400.0    1530700.0    1980700.0  
2016-08-15  1936689.0  1383600.0  772800.0    1636200.0    2280700.0  
2016-08-16  2305861.0  1206300.0  762700.0    1847400.0    2696900.0  
2016-08-17  2467495.0  1411700.0  950200.0    1893000.0    3027600.0  
...               ...        ...       ...          ...          ...  
2021-08-05  1109200.0  1734400.0  248300.0  107490000.0  214920000.0  
2021-08-06  1211400.0  1605800.0  205700.0  164120000.0  128260000.0  
2021-08-09   950800.0  1715100.0  128

In [117]:
sp_data.to_csv

In [156]:
def calcRSI (ticker):
    data = yf.download(ticker, period="5y", groupby="Ticker")
    data["Moving_Avg_200_Days"] = data["Adj Close"].rolling(window=200).mean()
    data["Price_Change"] = data["Adj Close"].pct_change()
    data["Gains"] = data["Price_Change"].apply(lambda x: x if x > 0 else 0)
    data["Losses"] = data["Price_Change"].apply(lambda x: abs(x) if x < 0 else 0)
    data["Avg_Up"] = data["Gains"].ewm(span=19).mean()
    data["Avg_Down"] = data["Losses"].ewm(span=19).mean()
    data = data.dropna()
    data["RS"] = data["Avg_Up"]/data["Avg_Down"]
    data["RSI"] = data["RS"].apply(lambda x: 100 - (100/(x+1)))
    return data

In [154]:
def calcRSI (ticker):
    data = yf.download(ticker, period="5y", groupby="Ticker")
    data["Moving_Avg_200_Days"] = data["Adj Close"].rolling(window=200).mean()
    data["Price_Change"] = data["Adj Close"].pct_change()
    temp1 = data['Price_Change']
    data['Gains'] = temp1.apply(lambda x: x if x > 0 else 0)
#     data["Gains"] = data.iloc[:,7].apply(lambda x: x if x > 0 else 0)
#     data["Losses"] = data.iloc[:,7].apply(lambda x: abs(x) if x < 0 else 0)
    data["Avg_Up"] = data["Gains"].ewm(span=19).mean()
#     data["Avg_Down"] = data["Losses"].ewm(span=19).mean()
    data = data.dropna()
#     data["RS"] = data["Avg_Up"]/data["Avg_Down"]
#     data["RSI"] = data["RS"].apply(lambda x: 100 - (100/(x+1)))
    return data

In [155]:
calcRSI(sp500_tickers[0]).iloc[-1]

[*********************100%***********************]  1 of 1 completed


Open                   2.000000e+02
High                   2.017700e+02
Low                    1.993100e+02
Close                  2.015700e+02
Adj Close              2.015700e+02
Volume                 1.906198e+06
Moving_Avg_200_Days    1.849369e+02
Price_Change           1.164370e-02
Gains                  1.164370e-02
Avg_Up                 4.475197e-03
Name: 2021-08-11 00:00:00, dtype: float64

In [160]:
sp5_tickers= ['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN']

In [196]:
def TodaysRSI():
#     stockRSI = []
    count = 0
    join_data = pd.DataFrame()
    for stock in sp5_tickers:
        stockRSI = calcRSI(stock).iloc[-1].to_frame()
        
        join_data.append(stockRSI) 
#         count+=1
#         print(count)
#     return join_data
    return join_data

In [197]:
fig = TodaysRSI()

[*********************100%***********************]  1 of 1 completed


C:\Users\samwe\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\samwe\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


[*********************100%***********************]  1 of 1 completed


C:\Users\samwe\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\samwe\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


[*********************100%***********************]  1 of 1 completed


C:\Users\samwe\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\samwe\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


[*********************100%***********************]  1 of 1 completed


C:\Users\samwe\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\samwe\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


[*********************100%***********************]  1 of 1 completed


C:\Users\samwe\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\samwe\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [198]:
fig

""


In [126]:
aapl_data = yf.download('AAPL', period="1mo")
aapl_data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-07-12,146.210007,146.320007,144.000000,144.500000,144.283829,76299700
2021-07-13,144.029999,147.460007,143.630005,145.639999,145.422119,100827100
2021-07-14,148.100006,149.570007,147.679993,149.149994,148.926865,127050800
2021-07-15,149.240005,150.000000,147.089996,148.479996,148.257874,106820300
2021-07-16,148.460007,149.759995,145.880005,146.389999,146.170990,93251400
2021-07-19,143.750000,144.070007,141.669998,142.449997,142.236893,121434600
2021-07-20,143.460007,147.100006,142.960007,146.149994,145.931351,96350000
2021-07-21,145.529999,146.130005,144.630005,145.399994,145.182480,74993500
2021-07-22,145.940002,148.199997,145.809998,146.800003,146.580383,77338200


In [140]:
aapl_data['Ticker'] = 'AAPL'
aapl_data

,Open,High,Low,Close,Adj Close,Volume,Ticker
Date,,,,,,,
2021-07-12,146.210007,146.320007,144.000000,144.500000,144.283829,76299700,AAPL
2021-07-13,144.029999,147.460007,143.630005,145.639999,145.422119,100827100,AAPL
2021-07-14,148.100006,149.570007,147.679993,149.149994,148.926865,127050800,AAPL
2021-07-15,149.240005,150.000000,147.089996,148.479996,148.257874,106820300,AAPL
2021-07-16,148.460007,149.759995,145.880005,146.389999,146.170990,93251400,AAPL
2021-07-19,143.750000,144.070007,141.669998,142.449997,142.236893,121434600,AAPL
2021-07-20,143.460007,147.100006,142.960007,146.149994,145.931351,96350000,AAPL
2021-07-21,145.529999,146.130005,144.630005,145.399994,145.182480,74993500,AAPL
2021-07-22,145.940002,148.199997,145.809998,146.800003,146.580383,77338200,AAPL


In [127]:
wmt_data = yf.download("WMT", period="1mo")
wmt_data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-07-12,140.690002,140.789993,139.330002,140.050003,140.050003,6505000
2021-07-13,139.990005,141.179993,139.940002,140.580002,140.580002,6315900
2021-07-14,140.710007,141.800003,139.830002,141.550003,141.550003,6270700
2021-07-15,142.000000,142.639999,141.089996,141.660004,141.660004,5991600
2021-07-16,141.699997,142.130005,140.949997,141.559998,141.559998,6031800
2021-07-19,141.410004,142.979996,139.770004,141.229996,141.229996,9127700
2021-07-20,140.970001,142.470001,140.779999,141.869995,141.869995,6340100
2021-07-21,142.500000,142.539993,140.710007,141.169998,141.169998,6197300
2021-07-22,141.130005,142.119995,140.800003,141.270004,141.270004,4337800


In [137]:
combined = pd.merge(aapl_data, wmt_data, how="left", on=['Date'])

In [138]:
combined

,Open_x,High_x,Low_x,Close_x,Adj Close_x,Volume_x,Open_y,High_y,Low_y,Close_y,Adj Close_y,Volume_y
Date,,,,,,,,,,,,
2021-07-12,146.210007,146.320007,144.000000,144.500000,144.283829,76299700,140.690002,140.789993,139.330002,140.050003,140.050003,6505000
2021-07-13,144.029999,147.460007,143.630005,145.639999,145.422119,100827100,139.990005,141.179993,139.940002,140.580002,140.580002,6315900
2021-07-14,148.100006,149.570007,147.679993,149.149994,148.926865,127050800,140.710007,141.800003,139.830002,141.550003,141.550003,6270700
2021-07-15,149.240005,150.000000,147.089996,148.479996,148.257874,106820300,142.000000,142.639999,141.089996,141.660004,141.660004,5991600
2021-07-16,148.460007,149.759995,145.880005,146.389999,146.170990,93251400,141.699997,142.130005,140.949997,141.559998,141.559998,6031800
2021-07-19,143.750000,144.070007,141.669998,142.449997,142.236893,121434600,141.410004,142.979996,139.770004,141.229996,141.229996,9127700
2021-07-20,143.460007,147.100006,142.960007,146.149994,145.931351,96350000,140.970001,142.470001,140.779999,141.869995,141.869995,6340100
2021-07-21,145.529999,146.130005,144.630005,145.399994,145.182480,74993500,142.500000,142.539993,140.710007,141.169998,141.169998,6197300
2021-07-22,145.940002,148.199997,145.809998,146.800003,146.580383,77338200,141.130005,142.119995,140.800003,141.270004,141.270004,4337800


In [ ]:
""